In [2]:
import pandas as pd  # таблицы

from transformers import MBartTokenizer, MBartForConditionalGeneration
from itertools import combinations
import torch

from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
import networkx as nx


dir = "D:\Develop\VS\HSE\Pump_Dump\\"  # папка проекта
main_df = pd.read_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|", index_col=0)
main_df.head()


,CHANNEL,NEWS_DATE,NEWS_TEXT,WORDS,TICKERS,WORDS_lemma,WAPRICE,Date,Price,Open,...,Change_A,Mood_0,Mood_1,Mood_2,Mood_B_0,Mood_B_1,Mood_B_2,Mood_A_0,Mood_A_1,Mood_A_2
0,if_stocks,2024-06-05,Российских инвесторов скоро засыплет дивиденда...,российских инвесторов скоро засыплет дивиденда...,NaN,российский инвестор скоро засыпать дивиденд тр...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,if_stocks,2024-06-05,20% 25%. Дивов не будет?! С 1 января 2025 года...,дивов января налог прибыль вырастет выиграет п...,NaN,див январь налог прибыль вырасти выиграть прои...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,if_stocks,2024-06-05,Рынки утром Коррекция на Мосбирже пошла на спа...,рынки утром коррекция мосбирже пошла спад каки...,"gazp,gaza",рынок утро коррекция мосбирже пойти спад какой...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,if_stocks,2024-06-04,Индекс Мосбиржи падает третью неделю. Мы дости...,индекс мосбиржи падает третью неделю достигли ...,posi,индекс мосбиржи падать третий неделя достигнут...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,if_stocks,2024-06-04,Информация для новичков Профессионалам на заме...,информация новичков профессионалам заметку кан...,"sber,astr,lkoh",информация новичок профессионал заметка канал ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)

In [4]:
model = MBartForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [5]:
def gen_summary(article_text):
    input_ids = tokenizer(
        [article_text],
        max_length=600,
        truncation=True,
        return_tensors="pt",
    )["input_ids"].to(device)
    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]
    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

In [ ]:
main_df["Summary_Bart"] = main_df["NEWS_TEXT"].apply(gen_summary)

In [7]:
# Обрабатываем текст на основе PageRank

def similarity(s1, s2):
    if not len(s1) or not len(s2):
        return 0.0
    return len(s1.intersection(s2))/(1.0 * (len(s1) + len(s2)))

# Выдает список предложений отсортированных по значимости
def textrank(text):
    sentences = sent_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    lmtzr = RussianStemmer()
    words = [set(lmtzr.stem(word) for word in tokenizer.tokenize(sentence.lower()))
             for sentence in sentences] 	 
    pairs = combinations(range(len(sentences)), 2)
    scores = [(i, j, similarity(words[i], words[j])) for i, j in pairs]
    scores = filter(lambda x: x[2], scores)
    g = nx.Graph()
    g.add_weighted_edges_from(scores)
    pr = nx.pagerank(g)
    return sorted(((i, pr[i], s) for i, s in enumerate(sentences) if i in pr), key=lambda x: pr[x[0]], reverse=True)

# Сокращает текст до нескольких наиболее важных предложений
def sumextract(text, n=5):
    tr = textrank(text)
    top_n = sorted(tr[:n])
    return ' '.join(x[2] for x in top_n)

In [8]:
main_df["Summary_PageRank"] = main_df["NEWS_TEXT"].apply(sumextract)

In [ ]:
main_df

In [9]:
# main_df[["NEWS_TEXT","Summary_Bart","Summary_PageRank"]].to_html(dir+"Answers\\compare.html")
# main_df.to_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|")
main_df.to_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|")